In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(reshape2))
suppressMessages(library(gridExtra))
suppressMessages(library(grid))
suppressMessages(library(rstatix))
suppressMessages(library(ggpubr))

source('../../0_metadata/theme_CRP-MISC.R')

── Attaching packages ───────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter




## Biomarkers of damage

#### read in redcap

In [2]:
### read in data
mdf_raw = read.csv("/workdir/cjl332/prevail/revisions/PreVAILCDEs-PlasmaProteinMarkers_DATA_2022-10-26_0619.csv")

### manual fixing
mdf_raw <- mdf_raw %>% 
    filter(!(study_id %in% c(160,161) & redcap_repeat_instance ==2)) %>%
    mutate(siteid = toupper(siteid))


### Split and combine
not_all_na <- function(x) any(!is.na(x))

mdf1 <- mdf_raw %>% filter(redcap_repeat_instrument == "") %>% 
                    select(!c(redcap_repeat_instrument, redcap_repeat_instance))%>%
                    select(study_id,siteid)
#                     select_if(~ !any(is.na(.)))  #%>% colnames()

mdf2 <- mdf_raw %>% filter(redcap_repeat_instrument == "labs_and_microbiology") %>% 
                    select(!c(redcap_repeat_instrument, redcap_repeat_instance))%>%
                    select(!siteid)
#                     select_if(~ !any(is.na(.)))  #%>% colnames()


### combine (dont need mdf3)
mdf <- merge(mdf1, mdf2, by = "study_id") %>% rename(ldh_date = ldhdate,
                     ferritin_date = ferr_date,
                     alb_date = albumin_date,
                    alt_date = alamin_date)

mdf <- mdf[,!grepl("units",colnames(mdf))]

head(mdf)

,study_id,siteid,wbc,wbc_date,crp_date,crp,ldh,ldh_date,alt,trop,trop_date,ferritin,ferritin_date,bnp,bnp_date,creatinine,alb,alb_date,alt_date
,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,1,UCSF0114,28.7,2021-09-11,2021-09-14,175.4,696,2021-09-09,33,NA,,NA,,,,0.66,2.7,2021-09-06,2021-09-06
2,10,UCSF0039,8.7,,,NA,,,24,NA,,NA,,,,0.85,4.9,,
3,100,UCSF0015,5.0,,,NA,,,NA,NA,,NA,,,,12.16,,,
4,11,UCSF0066,8.8,,,NA,,,19,NA,,NA,,,,0.25,4.5,,
5,111,UCSF0153,6.5,,,NA,,,NA,NA,,NA,,,,1.48,,,
6,114,UCSF0156,5.1,,,NA,,,11,NA,,NA,,,,0.55,4.3,,


---
# cfRNA and wbRNA data

### cfRNA

In [3]:
##------------------------------------
# META DATA
meta_data <- read.delim("../../0_metadata/cfrna.tsv") %>% 
    filter(sample_group != -1)

##------------------------------------
# DECONVOLUTION RESULTS
results = read.csv("../../1_sample-output/cfrna_deconvolution.csv",row.names=1)

##------------------------------------
# MERGE
meta_data_too = merge(meta_data,results,by.x="cfrna_sample_id",by.y=0)

##------------------------------------
# MELT

IDVARS = c(colnames(meta_data))
df_melt <- melt(meta_data_too,id.vars=IDVARS)
i = length(IDVARS)
colnames(df_melt)[(i+1):(i+2)] <- c("celltype","fraction")


##---------------------------
# Relabel cell types

group = list("Myeloid Progenitor" = c("Myeloid.progenitor"),
             "Erythrocyte/Erythroid Progenitor" = c("Erythrocyte.erythroid.progenitor"),
             "Intestinal Epithelial" = c("Intestinal.tuft.cell","Intestinal.enterocyte"),
             "T Cell / NK Cell" = c("T.Cell","NK.Cell"),
            "Kidney Epithelial Cell" = c("Kidney.epithelial.cell"),
            "Endothelial Cell" = c("Endothelial.cell"),
             "Neutrophil" = c("neutrophil"),
             "Platelet" = c("Platelet"),
             "Monocyte" = c("Monocyte"),
             "Thymocyte" = c("Thymocyte"),
             'Intrahepatic cholangiocyte' = c('Intrahepatic.cholangiocyte')
            )

# create dataframe
df2 <- stack(group) 
colnames(df2) <- c("celltype","new_celltype")

# merge and fix nas
df_melt2 <- merge(df_melt,df2,all.x=TRUE)
df_melt2$new_celltype <- as.character(df_melt2$new_celltype)
df_melt_all <- df_melt2                                                     ### SAVE FOR LATER PLOTTING
df_melt2[is.na(df_melt2$new_celltype),"new_celltype"] <- "other"

# Save as main table
df_melt2$celltype <- df_melt2$new_celltype
df_melt2 <- df_melt2 %>% group_by(across(c(-fraction))) %>% summarize(fraction = sum(fraction))

df_melt <- df_melt2 %>% ungroup()


##------------------------------------
# MODIFY IDs
df_melt <- df_melt %>%
    select(cfrna_sample_id,Diagnosis,timepoint,PTID,Date,severity,celltype,fraction) %>%
    mutate(PTID = toupper(PTID))

df_melt_cf <- df_melt

`summarise()` has grouped output by 'celltype', 'unique_sample_id', 'PTID', 'Date', 'origin', 'Diagnosis',
'severity', 'timepoint', 'Diagnosis_info', 'Date.of.symptom.onset', 'hospitalization', 'hospital_onset',
'hospital_resolved', 'icu', 'icu_onset', 'icu_resolved', 'hosp_length', 'icu_length', 'samp_rel_hosp',
'samp_rel_hosp_resolved', 'samp_rel_icu', 'samp_rel_icu_resolved', 'samp_rel_sym', 'immunocompromised',
'immuno_comment', 'pcr_date', 'pcr_result_1positive_2negative', 'pcr_positive_MC_1yes', 'antibody_date',
'antibody_positive_MC_1yes', 'antibody_result_1positive_2negative', 'Fever_5_days_1yes',
'Conjunctivitis_1yes', 'Mucosal_changes_1yes', 'Rash_1yes', 'Extremity_changes_1yes',
'Large_cervical_lymphdenopathy_1yes', 'kd_like_MC_1yes', 'highflow_o2_1yes', 'noninvasive_ventilate_1yes',
'mechanical_ventilate_1yes', 'req_vasopressors_inotropes_MC_1yes', 'Echo.depressed.function_1yes',
'Echo.coronary.artery.aneurysms_1yes', 'respiratory_failure_MC_1yes', 'cardiac_abnormal_MC_1yes

### wbrna

In [4]:
##------------------------------------
# META DATA
meta_data <- read.delim("../../0_metadata/wbrna.tsv") %>% 
    filter(sample_group != -1) %>%
    rename(wbrna_sample_id = SEQ_ID..UCSFonly.)

##------------------------------------
# DECONVOLUTION RESULTS
results = read.csv("../../../3_wbRNA/deconvolution_new/prevail_bayes-prism.csv",row.names=1)

##------------------------------------
# MERGE
meta_data_too = merge(meta_data,results,by.x="wbrna_sample_id",by.y=0)

##------------------------------------
# MELT

IDVARS = c(colnames(meta_data))
df_melt <- melt(meta_data_too,id.vars=IDVARS)
i = length(IDVARS)
colnames(df_melt)[(i+1):(i+2)] <- c("celltype","fraction")


##---------------------------
# Relabel cell types

group = list("Myeloid Progenitor" = c("Myeloid.progenitor"),
             "Erythrocyte/Erythroid Progenitor" = c("Erythrocyte.erythroid.progenitor"),
             "Intestinal Epithelial" = c("Intestinal.tuft.cell","Intestinal.enterocyte"),
             "T Cell / NK Cell" = c("T.Cell","NK.Cell"),
            "Kidney Epithelial Cell" = c("Kidney.epithelial.cell"),
            "Endothelial Cell" = c("Endothelial.cell"),
             "Neutrophil" = c("neutrophil"),
             "Platelet" = c("Platelet"),
             "Monocyte" = c("Monocyte"),
             "Thymocyte" = c("Thymocyte"),
             'Intrahepatic cholangiocyte' = c('Intrahepatic.cholangiocyte')
            )

# create dataframe
df2 <- stack(group) 
colnames(df2) <- c("celltype","new_celltype")

# merge and fix nas
df_melt2 <- merge(df_melt,df2,all.x=TRUE)
df_melt2$new_celltype <- as.character(df_melt2$new_celltype)
df_melt_all <- df_melt2                                                     ### SAVE FOR LATER PLOTTING
df_melt2[is.na(df_melt2$new_celltype),"new_celltype"] <- "other"

# Save as main table
df_melt2$celltype <- df_melt2$new_celltype
df_melt2 <- df_melt2 %>% group_by(across(c(-fraction))) %>% summarize(fraction = sum(fraction))

df_melt <- df_melt2 %>% ungroup()

##------------------------------------
# MODIFY IDs
df_melt <- df_melt %>%
    select(wbrna_sample_id,Diagnosis,timepoint,PTID,Date,severity,celltype,fraction) %>%
    mutate(PTID = toupper(PTID))

df_melt_wb <- df_melt

`summarise()` has grouped output by 'celltype', 'PTID', 'Date', 'unique_sample_id', 'origin', 'Diagnosis',
'severity', 'timepoint', 'Diagnosis_info', 'Date.of.symptom.onset', 'hospitalization', 'hospital_onset',
'hospital_resolved', 'icu', 'icu_onset', 'icu_resolved', 'hosp_length', 'icu_length', 'samp_rel_hosp',
'samp_rel_hosp_resolved', 'samp_rel_icu', 'samp_rel_icu_resolved', 'samp_rel_sym', 'immunocompromised',
'immuno_comment', 'pcr_date', 'pcr_result_1positive_2negative', 'pcr_positive_MC_1yes', 'antibody_date',
'antibody_positive_MC_1yes', 'antibody_result_1positive_2negative', 'Fever_5_days_1yes',
'Conjunctivitis_1yes', 'Mucosal_changes_1yes', 'Rash_1yes', 'Extremity_changes_1yes',
'Large_cervical_lymphdenopathy_1yes', 'kd_like_MC_1yes', 'highflow_o2_1yes', 'noninvasive_ventilate_1yes',
'mechanical_ventilate_1yes', 'req_vasopressors_inotropes_MC_1yes', 'Echo.depressed.function_1yes',
'Echo.coronary.artery.aneurysms_1yes', 'respiratory_failure_MC_1yes', 'cardiac_abnormal_MC_1yes

---
# Saved Variables

### palettes

In [5]:
##---------------------------
# New palette - cfRNA

cf_palette = c('#0234C1','#740C33','#4b7a47','#FB927F','#5CFBF4','#DBB008','#86C35E','#4D5D0D','#2BBBB3','#B27DF2','#1362AB','#FFCCCB')

## REORDER BASED ON PREVELANCE
sum <- df_melt_cf %>% 
    filter(Diagnosis == "MIS-C" & timepoint == "acute") %>% 
    group_by(celltype) %>% summarize(mean=mean(fraction))

sum <- sum[order(sum$mean,decreasing=TRUE),]

names(cf_palette) <- sum$celltype

##---------------------------
# New palette - wbRNA

library(qualpalr)
num_cells = length(unique(df_melt_wb$celltype))
wb_palette = sample(rev(qualpal(num_cells, colorspace=list(h=c(0,360), s=c(0.3,1), l=c(0.2,0.8)))$hex))

## REORDER BASED ON PREVELANCE
sum <- df_melt_wb %>% 
    filter(Diagnosis == "MIS-C" & timepoint == "acute") %>% 
    group_by(celltype) %>% summarize(mean=mean(fraction))

sum <- sum[order(sum$mean,decreasing=TRUE),]

names(wb_palette) <- sum$celltype

##---------------------------
# New palette - wbRNA
group_palette = c("low" = "#FFF200", "high" = "#2E3192")

### functions

In [6]:
get_plot_df <- function(mdf, df_melt, MARKER, CUTOFF, ID_VAR, DATE_THRESH = 4){

    ##------------------------------------
    # If there is a date column associated with the protein
    if (paste0(MARKER,"_date") %in% colnames(mdf)){
        
        # filter samples
        mdf_pb = mdf %>%  
            select(siteid, .data[[MARKER]], paste0(MARKER,"_date")) %>% 
            filter(.data[[MARKER]] != "")

        mdf_pb[[MARKER]] <- as.numeric(mdf_pb[[MARKER]])

        # merge w/ TOO and filter by date
        mdf_pb <- merge(mdf_pb, df_melt, by.x = "siteid", by.y="PTID") %>%
                mutate(date_diff = as.Date(Date) - as.Date(.data[[paste0(MARKER,"_date")]])) %>%
                filter(date_diff <= DATE_THRESH)
        }else{
        
        print("NO DATE")
        # filter samples
        mdf_pb = mdf %>% 
            select(siteid, .data[[MARKER]]) %>% 
            filter(.data[[MARKER]] != "")

        mdf_pb[[MARKER]] <- as.numeric(mdf_pb[[MARKER]])

        # merge w/ TOO
        mdf_pb <- merge(mdf_pb, df_melt, by.x = "siteid", by.y="PTID") %>% filter(timepoint == "acute")
    }
    
    
    ##------------------------------------
    # add universal id variable
    mdf_pb$sample_id <- mdf_pb[[ID_VAR]]
    
    ##------------------------------------
    # add "high" and "low" markers
    mdf_pb <- mdf_pb %>%
            mutate(group = ifelse(.data[[MARKER]] >= CUTOFF, "high","low"))
    
    return(mdf_pb)
}

plot_protein_too <- function(mdf, df_melt, MARKER, CUTOFF, val_vector, ID_VAR, DATE_THRESH = 4){
    
    ##------------------------------------
    # Get data
    mdf_pb <- get_plot_df(mdf, df_melt, MARKER, CUTOFF, ID_VAR, DATE_THRESH)
    
    ##------------------------------------
    # Plot distribution of values
    hist_plt <- mdf_pb %>% select(.data[[MARKER]],sample_id) %>% unique() %>% # nrow()
        ggplot(aes(.data[[MARKER]])) +
        geom_histogram(bins = 50) +
        theme(text = element_text(size = 18))+
        geom_vline(xintercept = CUTOFF, linetype= "dashed", color = "red", size = 2, alpha = 0.5)
    
    ##------------------------------------
    # Create conf table
    tmp <- mdf_pb %>% select(sample_id, Diagnosis, group) %>% unique() %>% mutate(group = factor(group, levels = c("low","high")))
    conf_table <- table(tmp$Diagnosis, tmp$group, useNA = "ifany") %>% tableGrob()
        
    ##------------------------------------
    # Make stacked bar plot
    too_sum <- mdf_pb %>% 
            group_by(group,celltype) %>% summarize(mean_fraction = mean(fraction)) %>% ungroup() %>%
            mutate(group = factor(group, levels = c("low","high")))
    
    labels = names(val_vector)

    grouped_plot <- too_sum %>%
        mutate(celltype=factor(celltype,levels=labels)) %>%
        ggplot(aes(x=group,y=mean_fraction,fill=celltype))+
        geom_bar(position="stack",stat="identity", colour="black",size = 0.15)+
        theme(plot.margin=grid::unit(c(1,0,0,-0.9), "mm"),
             legend.position = "bottom",
             text = element_text(size = 18)
                )+
        scale_fill_manual(values=val_vector)+
      guides(fill=guide_legend(nrow=4, byrow=TRUE))+
        labs(title = gsub("_.*","",ID_VAR), y = "fraction")
    
    legend <- cowplot::get_legend(grouped_plot)
    
    grouped_plot <- grouped_plot + theme(legend.position = "none")

    return(list("conf_table" = conf_table,
             "hist_plt" = hist_plt,
            "grouped_plot" = grouped_plot,
               "legend" = legend))
}



plot_cto <- function(mdf, df_melt, MARKER, CUTOFF, val_vector, ID_VAR, CELLTYPE, DATE_THRESH = 4){
    
    ##------------------------------------
    # Get data
    mdf_pb <- get_plot_df(mdf, df_melt, MARKER, CUTOFF, ID_VAR, DATE_THRESH) %>% 
        filter(celltype == CELLTYPE)
        
    boxplot <- mdf_pb %>% 
        mutate(group = factor(group, levels = c("low","high"))) %>%
        ggplot()  + 
            geom_boxplot(aes(x=group,y=fraction,fill=group),outlier.shape = NA,size = 0.2, color = "black")+ #, color = "black"
            geom_jitter(aes(x=group,y=fraction),width=.2,height=0,size = 0.01)+
            theme_prevail()+
    #         scale_fill_manual(values=expGroupPalette) + 
            theme(panel.grid.minor = element_blank())

    return(boxplot)
}

paired_boxplot <- function(df_melt_cf, df_melt_wb, MARKER, CUTOFF, CELLTYPE){

    mdf_cf <- get_plot_df(mdf, df_melt_cf, MARKER, CUTOFF, "cfrna_sample_id") %>% 
        select(!cfrna_sample_id) %>%
        mutate(analyte = "cfrna")

    mdf_wb <- get_plot_df(mdf, df_melt_wb, MARKER, CUTOFF, "wbrna_sample_id") %>% 
        select(!wbrna_sample_id) %>%
        mutate(analyte = "wbrna")
    
    mdf <- rbind(mdf_cf, mdf_wb) %>% filter(celltype == CELLTYPE) %>% unique()
    
    stat.test <- mdf %>%
    group_by(analyte) %>%
    wilcox_test(fraction ~ group, paired = FALSE) %>%
    add_xy_position(x="analyte", dodge = 0.8)


    boxplot <- mdf %>% 
            mutate(group = factor(group, levels = c("low","high"))) %>%
            mutate(analyte = factor(analyte, levels = c("cfrna","wbrna"))) %>%
            ggplot()  + 
            geom_boxplot(aes(x=analyte,y=fraction,fill=group),outlier.shape = NA,size = 0.2, color = "black")+ #, color = "black"
            geom_point(aes(x = analyte, y= fraction, fill=group),
                       position = position_jitterdodge(jitter.width=0.1,seed=100,jitter.height=0),
                       size = 0.5,alpha= 0.5)+       # ,pch=21 ,color = "black"
    #         theme_prevail()+
    #         scale_fill_manual(values=expGroupPalette) + 
            theme(panel.grid.minor = element_blank(),
                 text = element_text(size = 18),
                 plot.margin=grid::unit(c(1,0,0,-0.9), "mm"),
                axis.title.y = element_blank())+
            labs(title=CELLTYPE)

    boxplot <- boxplot + stat_pvalue_manual(stat.test, label = "p", tip.length = 0)

    return(boxplot)
}

paired_boxplot_publish <- function(df_melt_cf, df_melt_wb, MARKER, CUTOFF, CELLTYPE, group_palette){

    mdf_cf <- get_plot_df(mdf, df_melt_cf, MARKER, CUTOFF, "cfrna_sample_id") %>% 
        select(!cfrna_sample_id) %>%
        mutate(analyte = "cfrna")

    mdf_wb <- get_plot_df(mdf, df_melt_wb, MARKER, CUTOFF, "wbrna_sample_id") %>% 
        select(!wbrna_sample_id) %>%
        mutate(analyte = "wbrna")
    
    mdf <- rbind(mdf_cf, mdf_wb) %>% filter(celltype == CELLTYPE) %>% unique()
    
    stat.test <- mdf %>%
    group_by(analyte) %>%
    wilcox_test(fraction ~ group, paired = FALSE) %>% 
    add_significance("p") %>%
    add_xy_position(x="analyte", dodge = 0.8)

    boxplot <- mdf %>% 
            mutate(group = factor(group, levels = c("low","high"))) %>%
            mutate(analyte = factor(analyte, levels = c("cfrna","wbrna"))) %>%
            ggplot()  + 
            geom_boxplot(aes(x=analyte,y=fraction,fill=group),outlier.shape = NA,size = 0.2, color = "black")+ #, color = "black"
            geom_point(aes(x = analyte, y= fraction, fill=group),
                       position = position_jitterdodge(jitter.width=0.2,seed=100,jitter.height=0),
                       size = 0.5)+       # ,pch=21 ,color = "black"
            theme_prevail()+
            scale_fill_manual(values = group_palette) + 
            theme(panel.grid.minor = element_blank(),
                 text = element_text(size = 18),
                 plot.margin=grid::unit(c(1,1,1,1), "mm"),
                axis.title.y = element_blank(),
                 axis.title.x = element_blank(),
                 title = element_blank())

    boxplot <- boxplot + stat_pvalue_manual(stat.test, label = "p", tip.length = 0, label.size = 2)

    return(boxplot)
}

In [7]:
make_all_plots <- function(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, cf_palette, wb_palette,group_palette){

    cfrna <- plot_protein_too(mdf, df_melt_cf, MARKER, CUTOFF,
                            cf_palette, "cfrna_sample_id", DATE_THRESH = 4)

    wbrna <- plot_protein_too(mdf, df_melt_wb, MARKER, CUTOFF,
                              cf_palette, "wbrna_sample_id", DATE_THRESH = 4)

    ##------------------------------------
    # Comparison w/ histograms
    options(repr.plot.width =15, repr.plot.height= 8)

    lay = rbind(c(1,1,5,5,6,6),
                c(3,3,5,5,6,6),
                c(2,2,5,5,6,6),
                c(4,4,7,7,7,7))

    comp_hist <- grid.arrange(cfrna$conf_table,wbrna$conf_table,
                 cfrna$hist_plt, wbrna$hist_plt,
                 cfrna$grouped_plot, wbrna$grouped_plot,
                 cfrna$legend,
                 layout_matrix = lay, 
                 top = textGrob(MARKER,gp=gpar(fontsize=30,font=1)))

    ##------------------------------------
    # Comparison w/ boxplots

    boxplot <- paired_boxplot(df_melt_cf, df_melt_wb, MARKER, CUTOFF, CELLTYPE)


    options(repr.plot.width =15, repr.plot.height= 8)

    lay = rbind(c(1,1,2,2,4,4,4,4),
                c(1,1,2,2,4,4,4,4),
                c(1,1,2,2,4,4,4,4),
                c(3,3,3,3,3,3,3,3))

    grid.arrange(cfrna$grouped_plot, wbrna$grouped_plot+theme(axis.text.y = element_blank(),
                                                              axis.title.y = element_blank(),
                                                             axis.ticks.y = element_blank()),
                 cfrna$legend,
                 boxplot,
                 layout_matrix = lay, 
                 top = textGrob(MARKER,gp=gpar(fontsize=30,font=1)))
    
    
    publ_plt <- paired_boxplot_publish(df_melt_cf, df_melt_wb, MARKER, CUTOFF, CELLTYPE, group_palette)
    
    return(publ_plt)
}

---
# publish figures

In [12]:
##--------------------------
# pdf(file=paste0("plots/panelF_wbc-platelet.pdf"),
#     width=WIDTH,height=HEIGHT, paper="special", bg="white",
#     fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

png(file = "tmp.tmp")

MARKER = "crp"
CUTOFF = 18
CELLTYPE = "Endothelial Cell"
crp_plt <- make_all_plots(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, cf_palette, wb_palette, group_palette)


MARKER = "wbc"
CUTOFF = 11
CELLTYPE = 'Platelet'
wbc_plt <- make_all_plots(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, cf_palette, wb_palette,group_palette)


MARKER = "alt"
CUTOFF = 36
CELLTYPE = 'Intrahepatic cholangiocyte'
alt_plt <- make_all_plots(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, cf_palette, wb_palette, group_palette)


MARKER = "alb"
CUTOFF = 3.4
CELLTYPE = 'Kidney Epithelial Cell'
alb_plt <- make_all_plots(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, cf_palette, wb_palette, group_palette)
dev.off()

system("rm tmp.tmp")

`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.
Warning message in get_plot_df(mdf, df_melt, MARKER, CUTOFF, ID_VAR, DATE_THRESH):
“NAs introduced by coercion”
`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.
Warning message in get_plot_df(mdf, df_melt, MARKER, CUTOFF, ID_VAR, DATE_THRESH):
“NAs introduced by coercion”
`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.
Warning message in get_plot_df(mdf, df_m

png 
  2

In [13]:
options(repr.plot.width = 8, repr.plot.height = 6)

WIDTH = 1.55
HEIGHT = 1.28


##--------------------------
pdf(file=paste0("plots/panelF_wbc-platelet.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 1.2
wbc_plt + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))

dev.off()
##--------------------------
pdf(file=paste0("plots/panelF_crp-endothelial.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 0.6
crp_plt + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))

dev.off()

WIDTH = 1.64
##--------------------------
pdf(file=paste0("plots/panelF_alb-kidney.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 0.021
alb_plt + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))

dev.off()
##--------------------------
pdf(file=paste0("plots/panelF_alt-liver.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 0.042
alt_plt + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))

dev.off()

png 
  2

png 
  2

png 
  2

png 
  2